In [33]:
!pip install pandas
!pip install requests
!pip install beautifulsoup4
!pip install tqdm
!pip install yahoo_fin
!pip install "git+https://github.com/tqdm/tqdm.git@devel#egg=tqdm"

In [5]:
import datetime
import pandas as pd # pip install pandas
import requests # pip install requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import yfinance as yf
from tqdm import tqdm
import numpy as np
from yahoo_fin import options
from tqdm import tqdm

In [6]:
headers = {
    'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
}
url = 'https://www.marketwatch.com/tools/earningscalendar'
r = requests.get(url, headers = headers)
# Request with url and header
soup = BeautifulSoup(r.content, 'html.parser')
tabpane = soup.find('div', 'tabpane')
earning_tables = tabpane.find_all('div', 'daypage')
current_datetime = datetime.date.today() + datetime.timedelta(days=1)
tomorrow = current_datetime.strftime('%b_%d')
anticipated_earnings = pd.DataFrame()
print('Preparing to Scrap Data...')
for earning_table in tqdm(earning_tables):
    earning_date = earning_table['id'].replace('page', '')
    earning_date = earning_date[:3] + '_' + earning_date[3:]
    if tomorrow == earning_date:
        table = earning_table.find("table")
        pretty = table.prettify()
        pd_table = pd.read_html(pretty)
        pd_table_1 = pd_table[0]
        anticipated_earnings = pd_table_1
print('Finished Scraping Data...')
anticipated_earnings.columns = ["Company Name", "Symbol", "Fiscal Quarter", "EPS Forecast", "EPS Actual", "Suprise"]
pd.set_option("display.max_rows", None, "display.max_columns", None)
anticipated_earnings = anticipated_earnings.drop(columns=["Suprise", "EPS Actual", "Fiscal Quarter","EPS Forecast"])
eligible_tickers = anticipated_earnings
print('Finished Processing Data...')

100%|██████████| 17/17 [00:00<00:00, 732.19it/s]

Preparing to Scrap Data...
Finished Scraping Data...
Finished Processing Data...


In [7]:
print(eligible_tickers)

                                      Company Name Symbol
0                               nVent Electric PLC    NVT
1                   AllianceBernstein Holding L.P.     AB
2                                  Cowen Inc. Cl A   COWN
3                      Sensient Technologies Corp.    SXT
4                           Lantheus Holdings Inc.   LNTH
5                                   ENI S.p.A. ADR      E
6                                   Andritz AG ADR  ADRZY
7                China Construction Bank Corp. ADR  CICHY
8              Bank of Communications Co. Ltd. ADR  BCMXY
9                      DBS Group Holdings Ltd. ADR  DBSDY
10           China Eastern Airlines Corp. Ltd. ADR    CEA
11                        Magna International Inc.    MGA
12        Banco Bilbao Vizcaya Argentaria S.A. ADR   BBVA
13                         Syneos Health Inc. Cl A   SYNH
14                                  Moog Inc. Cl B  MOG.B
15                          Bank of China Ltd. ADR  BACHY
16            

In [8]:
current_week = datetime.date.today().isocalendar()[1]
today_formatted = datetime.date.today().strftime("%Y-%m-%d")

In [9]:
raw_data = []
for index, row in tqdm(eligible_tickers.iterrows()):
    try:
        dates = options.get_expiration_dates(row['Symbol'])
        if len(dates) != 0:
            closest_options_week = datetime.datetime.strptime(dates[0], '%B %d, %Y').isocalendar()[1]
            if closest_options_week - current_week < 2:
                current_ticker = float(yf.download(row['Symbol'],today_formatted, progress=False)['Adj Close'])
                chain = options.get_options_chain(row['Symbol'],dates[0])["puts"]
                total_open_interest = chain["Open Interest"].replace('-','0').astype('int').sum()
                total_volume = chain["Volume"].replace('-','0').astype('int').sum()
                temp = [row['Symbol'], current_ticker , dates[0], total_open_interest, total_volume]
                raw_data.append(temp)
    except Exception as e: 
        print(e)


76it [00:59,  1.27it/s]


In [10]:
raw_data

[['AZN', 67.22000122070312, 'April 29, 2022', 5009, 4429],
 ['WY', 41.900001525878906, 'April 29, 2022', 847, 327],
 ['XOM', 87.19999694824219, 'April 29, 2022', 31888, 38167],
 ['BMY', 77.19999694824219, 'April 29, 2022', 11332, 8868],
 ['CL', 81.22000122070312, 'April 29, 2022', 3926, 8984],
 ['CHTR', 461.489990234375, 'April 29, 2022', 1289, 894],
 ['HON', 189.9199981689453, 'April 29, 2022', 4456, 5073],
 ['CVX', 161.7899932861328, 'April 29, 2022', 9707, 6079],
 ['ABBV', 156.30999755859375, 'April 29, 2022', 6325, 11212],
 ['CBOE', 115.2699966430664, 'April 29, 2022', 2931, 1420],
 ['PSX', 87.20999908447266, 'April 29, 2022', 1645, 836],
 ['TAL', 3.0299999713897705, 'April 29, 2022', 994, 595]]

In [11]:
final_stock_options = pd.DataFrame(raw_data)
final_stock_options.columns = ["Ticker", "Price", "Clos Exp", "Tot OI", "Volume"]
print(final_stock_options)

   Ticker       Price        Clos Exp  Tot OI  Volume
0     AZN   67.220001  April 29, 2022    5009    4429
1      WY   41.900002  April 29, 2022     847     327
2     XOM   87.199997  April 29, 2022   31888   38167
3     BMY   77.199997  April 29, 2022   11332    8868
4      CL   81.220001  April 29, 2022    3926    8984
5    CHTR  461.489990  April 29, 2022    1289     894
6     HON  189.919998  April 29, 2022    4456    5073
7     CVX  161.789993  April 29, 2022    9707    6079
8    ABBV  156.309998  April 29, 2022    6325   11212
9    CBOE  115.269997  April 29, 2022    2931    1420
10    PSX   87.209999  April 29, 2022    1645     836
11    TAL    3.030000  April 29, 2022     994     595


In [12]:
print('Sorted by stock price')
final_stock_options.sort_values(by=['Price'])

Sorted by stock price


,Ticker,Price,Clos Exp,Tot OI,Volume
11,TAL,3.030000,"April 29, 2022",994,595
1,WY,41.900002,"April 29, 2022",847,327
0,AZN,67.220001,"April 29, 2022",5009,4429
3,BMY,77.199997,"April 29, 2022",11332,8868
4,CL,81.220001,"April 29, 2022",3926,8984
2,XOM,87.199997,"April 29, 2022",31888,38167
10,PSX,87.209999,"April 29, 2022",1645,836
9,CBOE,115.269997,"April 29, 2022",2931,1420
8,ABBV,156.309998,"April 29, 2022",6325,11212
7,CVX,161.789993,"April 29, 2022",9707,6079


In [13]:
print('Sorted by Open Interest')
final_stock_options.sort_values(by=['Tot OI'], ascending=False)

Sorted by Open Interest


,Ticker,Price,Clos Exp,Tot OI,Volume
2,XOM,87.199997,"April 29, 2022",31888,38167
3,BMY,77.199997,"April 29, 2022",11332,8868
7,CVX,161.789993,"April 29, 2022",9707,6079
8,ABBV,156.309998,"April 29, 2022",6325,11212
0,AZN,67.220001,"April 29, 2022",5009,4429
6,HON,189.919998,"April 29, 2022",4456,5073
4,CL,81.220001,"April 29, 2022",3926,8984
9,CBOE,115.269997,"April 29, 2022",2931,1420
10,PSX,87.209999,"April 29, 2022",1645,836
5,CHTR,461.489990,"April 29, 2022",1289,894


In [15]:
filtered_stocks = final_stock_options
cash = 2932
eligible_price = float(cash / 100)
for index, row in filtered_stocks.iterrows():
    try:
        if (row['Price'] > eligible_price):
            filtered_stocks.drop(index, inplace=True)
    except Exception as e:
        print(e)

In [16]:
filtered_stocks

,Ticker,Price,Clos Exp,Tot OI,Volume
11,TAL,3.03,"April 29, 2022",994,595
